In [1]:
!pip install -U selenium
!pip install -U pandas
!pip install -U bs4

Requirement already up-to-date: selenium in c:\users\alexi\anaconda3\lib\site-packages (3.141.0)
Requirement already up-to-date: pandas in c:\users\alexi\anaconda3\lib\site-packages (1.0.3)
Requirement already up-to-date: bs4 in c:\users\alexi\anaconda3\lib\site-packages (0.0.1)


In [2]:
# Importing Packages:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import pandas as pd
from bs4 import BeautifulSoup

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

driver = webdriver.Chrome(options = options)

In [4]:
driver.get('https://www.renodepot.com/en/tools/1')
driver.refresh()

In [5]:
selectMTL = driver.find_element_by_id("department")
selectMTL.click();

In [6]:
page_source = driver.page_source

In [7]:
soup = BeautifulSoup(page_source, 'lxml')

In [33]:
products = soup.find_all('div', class_ = 'product_desc')
make = []
makeTool = []
deltaPrice = []
normPrice = []
salePrice = []

for product in products:
    make.append(product.find('div', class_ = 'product_manufacturer').get_text().strip())
    makeTool.append(product.find('div', class_ = 'product_name').get_text().strip())
    deltaPrice.append(product.find('div', class_ = 'rebate').get_text().strip())
    normPrice.append(product.find('span', class_ = 'linethrough').get_text().strip())
    salePrice.append(product.find('span', class_ = 'integer').get_text().strip())
    

In [34]:
product_df = pd.DataFrame(columns=['Make', 'Tool_Type', 'Regular_Price', 'Sale_Price', 'Rebate'])

product_df['Make'] = make
product_df['Tool_Type'] = makeTool
product_df['Regular_Price'] = normPrice
product_df['Sale_Price'] = salePrice
product_df['Rebate'] = deltaPrice

In [35]:
product_df.head()

,Make,Tool_Type,Regular_Price,Sale_Price,Rebate
0,METABO HPT,"Pneumatic Stripe Brad Nailer - 5/8""-2""",$179.00,89,Save $90.00
1,MAKITA,Set of 2 Cordless Tools 18V,$319.00,189,Save $130.00
2,MAKITA,Set of 2 Cordless Tools - 18 V Lithium-ion,$399.00,249,Save $150.00
3,BOSCH,"Small Angle Grinder - 4 1/2"" - Set of 2",$149.00,99,Save $50.00
4,BOSCH,Laser Measuring Tool - 100 ft,$89.99,59,Save $30.00
